In [1]:
import numpy as np
import pandas as pd

from UpdateDB_ref import Endpoint_annotation as ep


*** CompoundDB module found. Will check the synonyms table to resolve CAS. ***



In [2]:
ep_refactored = ep.Endpoint(host='gea', dbname='cii_v3_test_refactored_2', user='postgres', password='DBAdmin')

In [3]:
endpoints_ref = ep_refactored.get_endpoints()

In [4]:
ep_refactored.get_total_annotations_per_endpoint(endpoints_ref)

/home/emarch/miniconda3/envs/inditex/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,Endpoints,YES,Pending,NO,No information
0,cmr,1909.0,41.0,890.0,2832.0
1,pbt,87.0,0.0,746.0,4839.0
2,vpvb,97.0,0.0,746.0,4829.0
3,endocrine_disruptor,299.0,0.0,0.0,5373.0
4,sensitiser,133.0,129.0,0.0,2143.0
5,chem_id,0.0,0.0,0.0,0.0


In [5]:
endpoint_pos = pd.read_sql_query("""select pea.annotation_id, rn.names, pea.endpoint 
from positive_endpoint_annotations pea
left join regulation_names rn on rn.id = pea.annotation_id
order by pea.id """, ep_refactored.conn)

In [6]:
endpoint_dict = {'CMR':endpoint_pos.loc[endpoint_pos['endpoint'] == 'cmr','names'].tolist(),
                 'PBT':endpoint_pos.loc[endpoint_pos['endpoint'] == 'pbt','names'].tolist(),
                 'vPvB':endpoint_pos.loc[endpoint_pos['endpoint'] == 'vpvb','names'].tolist(),
                 'Endocrine Disruptor':endpoint_pos.loc[endpoint_pos['endpoint'] == 'endocrine disruptor','names'].tolist()}

In [7]:
chem_ids = ep_refactored.get_chem_id_dataframe()

In [15]:
subs = ep_refactored.get_substances()

In [24]:
len(chem_ids)

5672

In [22]:
len(subs)

5672

In [23]:
len(subs[['class_name','preferred_name','mol_formula']].drop_duplicates())

4611

In [14]:
len(chem_ids.loc[chem_ids['chem_type_id'] == 1, :])

4694

In [21]:
subs.loc[subs['preferred_name'] == "Hexabromo-1,1'-biphenyl"].sort_values(by='id')

,id,class_name,preferred_name,mol_formula
5598,274,Flame Retardants,"Hexabromo-1,1'-biphenyl",C12H4Br6
5599,5658,Flame Retardants,"Hexabromo-1,1'-biphenyl",C12H4Br6


In [30]:
subs.loc[subs['id'] > 4610, :].sort_values(by='id')

,id,class_name,preferred_name,mol_formula
76,4611,nan,"Benzenemethanol, alpha-[2-(methylamino)ethyl]-...",C10H15NO
2706,4612,nan,"Ethanesulfonic acid, 2-[[[2,5-dichloro-4-[(2-m...",C17H15Cl2N4NaO5S2
548,4613,nan,Phenyl isothiocyanate,C7H5NS
3532,4614,nan,Vanadyl sulfate,O5SV
620,4615,nan,((2-Methylphenoxy)methyl)oxirane,C10H12O2
1635,4616,nan,Bisphenol S\nsynonym match: BPS 1,C12H10O4S
5666,4617,nan,"Benzenesulfonic acid, 2,5-dichloro-4-[[2-(dibu...",C27H32Cl2N6Na2O6S2
522,4618,nan,2-(7-methoxynaphthalen-1-yl)ethan-1-amine hydr...,C13H16ClNO
5060,4619,nan,Tributyl citrate,C18H32O7
314,4620,nan,Dimethoxydimethylsilane\nsynonym match: JJQZDU...,C4H12O2Si


In [8]:
chem_ids_unique = chem_ids.loc[chem_ids['chem_type_id'] == 1, 'id'].unique()

In [9]:
%%time
chemid_endpoint = ep_refactored.get_annotations_for_chem_id(chem_ids_unique, endpoint_dict)

CPU times: user 2min 14s, sys: 1.97 s, total: 2min 16s
Wall time: 11min 48s


In [10]:
len(chemid_endpoint)

4694

In [11]:
chemid_endpoint.sort_values(by='chem_id').head()

,chem_id,CMR,PBT,vPvB,Endocrine Disruptor
24,1.0,YES,NO,NO,No information
25,2.0,No information,No information,No information,No information
26,5.0,YES,NO,NO,YES
27,7.0,YES,NO,NO,No information
28,9.0,YES,No information,No information,No information


In [11]:
len(endpoints_ref)

5672

In [12]:
endpoints_ref.sort_values(by='chem_id')

,id,cmr,pbt,vpvb,endocrine_disruptor,sensitiser,chem_id
0,0,YES,NO,NO,No information,YES,1
5080,5080,YES,NO,NO,No information,YES,2
5081,5081,YES,NO,NO,No information,YES,3
5082,5082,YES,NO,NO,No information,YES,4
1,1,YES,NO,NO,YES,No information,5
4661,4661,YES,NO,NO,YES,No information,6
2,2,YES,NO,NO,No information,No information,7
5446,5446,YES,NO,NO,No information,No information,8
3,3,YES,No information,No information,No information,No information,9
4764,4764,YES,No information,No information,No information,No information,10


### Here it should come the check part
After getting the endpoint annotations per substance, and at the same time we update the database with them, the check function should come.
Before updating the DB, the check function should see if there's a substance id in the table.
If no, adds the new substance with the annotations.
If there is, then it checks the annotations and applies the filters we've spoken

In [15]:
# ep.add_annotations_to_database(subs_endpoint)

### Total annotation check
This only counts the total number of annotations per endpoint we have

In [23]:
ep_refactored.get_total_annotations_per_endpoint(chemid_endpoint)

,Endpoints,YES,Pending,NO,No information
0,CMR,1362.0,39.0,0.0,3293.0
1,PBT,38.0,0.0,356.0,4300.0
2,vPvB,43.0,0.0,356.0,4295.0
3,Endocrine Disruptor,148.0,0.0,0.0,4546.0
